Chapitre 13 - Ecrire des tests
===

Si nous avons couvert l'écriture de tests dans d'autres cours et dans quelques discussions, la rédaction de test reste encore quelque peu un mystère. Et pour cause.

Les tests cherchent à vérifier un ensemble de fonctionnement :
- les blocs programmés, pris séparément, fonctionnent correctement
- ces blocs interagissent correctement les uns avec les autres
- (pour les applications graphiques) la partie graphique de l'application interagit correctement avec la partie "cachée"
- les modifications futures - y compris par des tiers - ne provoquent pas d'erreur

## Les tests en python 

### Le choix des armes

Nous avons vu pour l'instant une solution très basique pour faire ce genre de travail :

In [ ]:
def carre(x):
    """ Calcul un carré
    
    :param x: Nombre à mettre en puissance
    :type x: int
    :returns: Carré du nombre
    :rtype: int
    """
    return x**2  # On en profite pour découvrir la fonction `**` qui permet de mettre à la puisse qui suit. x**3 = x*x*x

assert carre(8) == 64
assert carre(3) == 9

Si tout fonctionne, `assert` ne nous dit rien. Si par hasard nous avons une erreur, la fonction `assert` va simplement créer une erreur et stopper le reste du code. Malheureusement, cela n'est pas très pratique à l'échelle d'une application. Par exemple, si un seul test ne fonctionne pas, `assert` ne nous avertira pas des autres erreurs, ce qui empêchera une vision d'ensemble.

Pour python, heureusement, il existe des librairies plus complètes :
- `unittest` qui est incluse par défaut dans python
- `nosetest` qui est installable via `pypi` et qui est rétro-compatible avec `unittest`
- `py.test` qui est moins verbeux que les deux précédents et utilise beaucoup `assert`

Dans le cadre de ce cours, nous verrons le premier **unittest**. Sachez cependant que ce choix peut varier suivant :
- les préférences locales de votre équipe
- le besoin d'options complexes

### `Unittest`

Unittest est très facile à utiliser. Nous devons: 
1. Créer une classe dérivée de `unittest.TestCase`
2. Créer des fonctions commencant par `test_`
3. Utiliser des termes vérifiant les égalités tels que `assertEqual(x,y, message=None)`, `assertGreaterThan()`, etc.

Prenons comme exemple notre fonction précédente

In [ ]:
import unittest

class TestCarre(unittest.TestCase):
    """ Test l'ensemble des fonctions pour carré """
    
    def test_calcul_correct(self):
        self.assertEqual(carre(8), 64)
        self.assertEqual(carre(3), 9)
        self.assertEqual(carre(-1), 1)
        for x in range(9):
            self.assertEqual(carre(x), carre(-x))
            
    def test_erreur_quand_non_numeric(self):
        with self.assertRaises(TypeError):
            # Ne fonctionnera que si l'erreur TypeError est lancée
            carre("Ca va pas marcher...")
            
    def test_accepte_decimaux(self):
        """Le carré d'un décimal devrait être bien calculé"""
        self.assertEqual(carre(0.1), 0.01)

On peut lancer ces tests directement en python (on verra cependant plus tard qu'on ne les lance que rarement comme cela):

In [ ]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

On voit ici 3 choses :
- une fonction peut contenir une ou plusieurs assertions
- certaines assertions comme `assertRaises` s'utilise surtout avec `with`
- quand un test échoue, des détails sont donnés. Ici on se rend compte que les calculs ne sont pas si faciles quand il est question de décimaux...

#### Les assertions de `unittest`

| Method                    	| Checks that                                                                  	|
|---------------------------	|------------------------------------------------------------------------------	|
| assertEqual(a,b)          	| a == b                                                                       	|
| assertNotEqual(a,b)       	| a != b                                                                       	|
| assertTrue(x)             	| bool(x) is True                                                              	|
| assertFalse(x)            	| bool(x) is False                                                             	|
| assertIs(a,b)             	| a is b                                                                       	|
| assertIsNot(a,b)          	| a is not b                                                                   	|
| assertIsNone(x)           	| x is None                                                                    	|
| assertIsNotNone(x)        	| x is not None                                                                	|
| assertIn(a,b)             	| a in b                                                                       	|
| assertNotIn(a,b)          	| a not in b                                                                   	|
| assertIsInstance(a,b)     	| isinstance(a, b)                                                             	|
| assertNotIsInstance(a,b)  	| not isinstance(a, b)                                                         	|
| assertAlmostEqual(a,b)    	| round(a-b, 7) == 0                                                           	|
| assertNotAlmostEqual(a,b) 	| round(a-b, 7) != 0                                                           	|
| assertGreater(a,b)        	| a > b                                                                        	|
| assertGreaterEqual(a,b)   	| a >= b                                                                       	|
| assertLess(a,b)           	| a < b                                                                        	|
| assertLessEqual(a,b)      	| a <= b                                                                       	|
| assertRegex(s,r)          	| r.search(s)                                                                  	|
| assertNotRegex(s,r)       	| not r.search(s)                                                              	|
| assertCountEqual(a,b)     	| a et b sont égaux : même nombre d'éléments et mêmes éléments quelque soit leur ordre 	|

#### Mais si on ne les lance pas comme ca...

Comment les lance-t-on ? Typiquement, les dossiers de développement ressembleront à ca : 

- Dossier de travail
    - dossier_application (Module principal)
    - tests (Module des tests)
    - docs (dossier avec la documentation )
    - run.py ou app.py ou autre nom qui fait sens (outil pour lancer l'application)
    - env (Environnement virtuel qui sera soigneusement ignoré dans un gitignore)
    - README.md
    - LICENSE.md
    - .gitignore ( [Exemple pour python](https://raw.githubusercontent.com/github/gitignore/master/Python.gitignore)

Dans ce cadre là, on fera généralement :

```shell
cd DOSSIER_DE_TRAVAIL
source env/bin/activate
python -m unittest discover tests # Où `tests` est le module qui contient les tests
```

## Concepts fondamentaux de la rédaction de test

### Tests unitaires, tests d'intégration

> Dans le test unitaire, on vérifie le bon fonctionnement d'une partie précise d'un logiciel ou d'une portion d'un programme (appelée « unité » ou « module ») ; dans le test d’intégration, chacun des modules indépendants du logiciel est assemblé et testé \[comme\] ensemble.
> https://fr.wikipedia.org/wiki/Test_d%27int%C3%A9gration

### `setUp` et `tearDown`

Les fonctions de mise en place et de destruction sont des fonctions qui seront lancées avant chacun des tests d'une classe. Elle permette par exemple de générer une application et de tester cette application plusieurs fois. Ou de rentrer des données dans une base de données.

Exemple en mi-pseudocode mi-python  :


```python
class TestMiseAJourRessources(TestCase):
    def setUp(self):
        self.application = gazetteer
        self.user = gazetteer.User.Johanna
        self.application.login(self.user)
        
    def tearDown(self):
        try:
            self.application.logout(self.user)
        except:
            pass
        
    def test_update(self):
        self.application.route.update_item(1).nom = "Rome"
        self.assertEqual(self.application.item(1).nom, "Rome")
            
    def test_erreur_non_loguee(self):
        self.application.logout(self.user)
        with self.assertRaises(Forbidden):
            self.application.route.update_item(1).nom = "Rome"
        self.assertEqual(self.application.item(1).nom, "Roma")
```

### Les `Fixtures`

Les `fixtures` sont des données pour une base de données temporaire qui permettront de vérifier la validité des tests.

### Les `Mocks`

Un mock est une fonction qui va venir remplacer une autre fonction capable d'échouer. Très souvent, ces `Mocks` viennent aider à tester des services externes tels que des APIs hébergées par d'autres sites qui pourraient ne pas apprécier d'être la source de tests.

Nous n'aborderons pas les mocks ici mais vous pouvez trouver quelques tutoriaux en ligne :
- http://python-mock-tutorial.readthedocs.io/en/latest/introduction.html
- https://realpython.com/blog/python/testing-third-party-apis-with-mocks

## Tests et Flask